<a href="https://colab.research.google.com/github/AndreyBuyanov/HPC.Lab1/blob/main/HPC_Lab1_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

In [ ]:
import os
os.environ['APACHE_SPARK_DISTR_NAME'] = 'spark-3.0.2-bin-hadoop2.7'
os.environ['APACHE_SPARK_URL'] = 'https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz'

In [ ]:
![ ! -d $APACHE_SPARK_DISTR_NAME ] \
   && wget -q $APACHE_SPARK_URL \
   && tar xf $APACHE_SPARK_DISTR_NAME.tgz \
   && rm -f $APACHE_SPARK_DISTR_NAME.tgz

In [ ]:
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.0.2-bin-hadoop2.7'

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master('local').appName("HPC.Lab1").getOrCreate()

In [ ]:
df = spark.read.csv(r"./drive/My Drive/TPU/HPC/Lab1/brooklyn_sales_map.csv", header=True)
df.show()

+---+--------+--------------------+-----------------------+---------+-----+----+--------+--------------+--------------------+----------------+--------+-----------------+----------------+-----------+---------+----------+----------+-----------------+----------------------+----------+----------+------------+---------+---+------+------+----------+-------+-------+--------+----------+----------+----------+---------+----------+--------+-------------------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-------+---------+---------+--------------------+-------+--------+-------+-------+----------+----------+----------+---------+----------+---------+----------+--------+---------+--------+----------+--------+--------+---------+---------+---+--------+----------+-------+--------+----------+---------+----------+---------+---------+----------+----------+--------------------+--------+--------+--------+-------+--------+--------+----------+-

In [ ]:
from pyspark.sql import functions as F

1. Найдите среднююстоимость жилья (sale_price) и выведите новую таблицу, содержащую две колонки – стоимость жилья и процент отклонения стоимости от среднего значения.

In [ ]:
df_sale_price = df.withColumn('sale_price_d', df['sale_price'].cast('double')).select('sale_price_d').orderBy(F.desc('sale_price_d'))
df_sale_price.show()

+------------+
|sale_price_d|
+------------+
|4.99401179E8|
|      3.45E8|
|       3.4E8|
|   2.76947E8|
|       2.4E8|
|    2.0502E8|
|     2.025E8|
|   1.85445E8|
|      1.71E8|
|      1.69E8|
|      1.65E8|
|      1.61E8|
|       1.6E8|
|      1.58E8|
|    1.5425E8|
|1.38106368E8|
|   1.35212E8|
|      1.35E8|
|     1.324E8|
|      1.25E8|
+------------+
only showing top 20 rows



In [ ]:
df2 = df['sale_price'].cast('double').alias('sale_price')
df2

Column<b'CAST(sale_price AS DOUBLE) AS `sale_price`'>

In [ ]:
df3 = df.select(F.col('sale_price').cast('double')).collect()[0][0]
df3

499401179.0

In [ ]:
df_col_avg = df.select( *[F.mean(c).alias(c) for c in df.columns])

In [ ]:
df_col_avg = df.select(*[F.mean(c).alias(c) for c in df.columns])
for col in df.columns:
    avg_val = df_col_avg.select(col).collect()[0][0]
    print(avg_val)
    if avg_val == None:
        continue
    df.fillna(avg_val, subset=col)

195442.0
3.0
3009.0
None
1.6015349828238465
4275.651169275717
272.50998636420616
None
None
73.86666666666666
292.0267628832847
11208.429087476303
2.4861377957086903
0.17414162294088
2.673227027013198
2368.4637986302805
2970.8766408362603
1701.6663067976863
1.642660335701476
None
506754.47769281344
None
2009.4782428501624
None
309.78321063583206
492.221199791758
1971.0651604052691
19.08980269223677
40.888521967023124
11207.063718195228
None
71.83001237949745
34.460293420428805
5758.185060317126
3.0003264762331896
9.77639016218284
3.2102842809364547
None
None
None
None
None
None
None
None
None
None
None
None
None
2.188790628685446
0.007049070220723806
None
889.0
8459.415677843333
23612.377373834483
1207.2916984933888
22216.569591871674
158.4348956961492
274.3297325238371
248.9965067428752
115.6844215877364
173.28526510562082
209.22560646367802
2.086037507243323
1.3148244481904863
3.1021660498867436
22.08292287836485
22.347620239161355
53.08684349154378
109.69502729417162
39.3995667505134

In [ ]:
avg_sale_price = df_sale_price.groupBy().agg(F.mean('sale_price_d').alias('mean')).collect()
avg_sale_price_val = avg_sale_price[0]['mean']
print(avg_sale_price_val)

506754.47769281344


In [ ]:
df_sale_price_with_avg = df_sale_price.withColumn('sale_price_deviation', (df_sale_price.sale_price_d - avg_sale_price_val)/df_sale_price.sale_price_d * 100)
df_sale_price_with_avg.show()

+------------+--------------------+
|sale_price_d|sale_price_deviation|
+------------+--------------------+
|4.99401179E8|   99.89852757682559|
|      3.45E8|     99.853114644147|
|       3.4E8|   99.85095456538446|
|   2.76947E8|   99.81702113484066|
|       2.4E8|   99.78885230096132|
|    2.0502E8|   99.75282680826612|
|     2.025E8|   99.74975087521342|
|   1.85445E8|    99.7267359714779|
|      1.71E8|   99.70365235222643|
|      1.69E8|   99.70014527947171|
|      1.65E8|   99.69287607412556|
|      1.61E8|    99.6852456660293|
|       1.6E8|   99.68327845144199|
|      1.58E8|   99.67926931791594|
|    1.5425E8|   99.67147197556382|
|1.38106368E8|   99.63306943406634|
|   1.35212E8|   99.62521486429252|
|      1.35E8|   99.62462631282014|
|     1.324E8|   99.61725492621389|
|      1.25E8|   99.59459641784575|
+------------+--------------------+
only showing top 20 rows



2. Выведите таблицу, содержащую все категории класса зданий (building_class_category) и количество записей, которые к ним относятся.

In [ ]:
df_bcc = df.select('building_class_category')
df_bcc.show()

+-----------------------+
|building_class_category|
+-----------------------+
|   28  COMMERCIAL CO...|
|   29  COMMERCIAL GA...|
|   21  OFFICE BUILDINGS|
|    22  STORE BUILDINGS|
|        26 OTHER HOTELS|
|   31  COMMERCIAL VA...|
|    21 OFFICE BUILDINGS|
|   08  RENTALS - ELE...|
|   29  COMMERCIAL GA...|
|         30  WAREHOUSES|
|   41 TAX CLASS 4 - ...|
|   31  COMMERCIAL VA...|
|      11A CONDO-RENTALS|
|    21 OFFICE BUILDINGS|
|   28  COMMERCIAL CO...|
|        26 OTHER HOTELS|
|   31  COMMERCIAL VA...|
|   08  RENTALS - ELE...|
|   29  COMMERCIAL GA...|
|    21 OFFICE BUILDINGS|
+-----------------------+
only showing top 20 rows



In [ ]:
df_bcc_count = df_bcc.groupBy('building_class_category').count()
df_bcc_count.show()

+-----------------------+------+
|building_class_category| count|
+-----------------------+------+
|   32  HOSPITAL AND ...|   318|
|         17 CONDO COOPS|    29|
|   49 CONDO WAREHOUS...|    24|
|   11  SPECIAL CONDO...|     4|
|            17  CONDOPS|   448|
|      44  CONDO PARKING|  2596|
|    01 ONE FAMILY HOMES| 54205|
|   31 COMMERCIAL VAC...|   132|
|   48  CONDO TERRACE...|    90|
|   03 THREE FAMILY H...| 38691|
|   16  CONDOS - 2-10...|   439|
|    02 TWO FAMILY HOMES|106127|
|           27 FACTORIES|    84|
|          30 WAREHOUSES|   133|
|                   null|    83|
|   28 COMMERCIAL CONDOS|     7|
|   47 CONDO NON-BUSI...|   100|
|   36 OUTDOOR RECREA...|     4|
|   05  TAX CLASS 1 V...|  7377|
|   15  CONDOS - 2-10...|  8913|
+-----------------------+------+
only showing top 20 rows



3. Выведите таблицу, содержащую средние значения по каждому столбцу в датафрейме.

In [ ]:
df_col_avg = df.select(*[F.mean(c).alias(c) for c in df.columns])
df_col_avg.show()

+--------+--------+------------+-----------------------+------------------+-----------------+------------------+--------+--------------+-----------------+-----------------+------------------+------------------+----------------+-----------------+------------------+------------------+------------------+-----------------+----------------------+------------------+---------+------------------+---------+------------------+----------------+------------------+-----------------+------------------+------------------+--------+-----------------+------------------+-----------------+------------------+----------------+------------------+---------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-----------------+--------------------+---------+---------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+---

In [ ]:
df_for_sample = df.select(['neighborhood', 'building_class_category', 'gross_sqft', 'sale_price']).limit(5)
df_for_sample.show()

+--------------------+-----------------------+----------+----------+
|        neighborhood|building_class_category|gross_sqft|sale_price|
+--------------------+-----------------------+----------+----------+
|  DOWNTOWN-METROTECH|   28  COMMERCIAL CO...|         0| 499401179|
|DOWNTOWN-FULTON F...|   29  COMMERCIAL GA...|         0|  3.45e+08|
|    BROOKLYN HEIGHTS|   21  OFFICE BUILDINGS|    304650|   3.4e+08|
|          MILL BASIN|    22  STORE BUILDINGS|   2548000| 276947000|
|    BROOKLYN HEIGHTS|        26 OTHER HOTELS|    356000| 202500000|
+--------------------+-----------------------+----------+----------+

